In [1]:
import tensorflow as tf
import numpy as np
from functools import reduce
import sys

In [2]:
n_classes = 10
max_length = 10
n_instances = sum([reduce(lambda x, y: x*y,
                          range(n_classes,n_classes-length,-1)) for length in range(2, max_length+1)
])
n_instances

9864090

The following `X` will be our dataset (including training/validation/test sets).

In [3]:
help(sys.getsizeof)

Help on built-in function getsizeof in module sys:

getsizeof(...)
    getsizeof(object, default) -> int
    
    Return the size of object in bytes.



In [4]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       9.9Gi       4.1Gi       578Mi       1.5Gi       4.7Gi
Swap:           31Gi       2.7Gi        29Gi


In [5]:
X = np.zeros((n_instances, max_length, n_classes), dtype=np.float32)

In [6]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       9.9Gi       4.1Gi       584Mi       1.6Gi       4.6Gi
Swap:           31Gi       2.7Gi        29Gi


In [7]:
sys.getsizeof(X)

3945636128

`3.9` billion bytes! That's more than `3GB`. Let's verify this number.

In [8]:
n_instances * max_length * n_classes * (32//8)

3945636000

In [9]:
del X

In [10]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       9.9Gi       4.1Gi       586Mi       1.6Gi       4.6Gi
Swap:           31Gi       2.7Gi        29Gi


About right: The numbers are quite consistent.

<s>By contrast, it seems that `tf.zeros` does not allocate the memory immediately, taking only a memory of `184` bytes.</s>

In [11]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       9.9Gi       4.1Gi       587Mi       1.6Gi       4.6Gi
Swap:           31Gi       2.7Gi        29Gi


In [12]:
X = tf.zeros((n_instances, max_length, n_classes), dtype=tf.float32)
# float32 unable to be allocated on 4GB-RAM X61s whereas int8 can.
#X = tf.zeros((n_instances, max_length, n_classes), dtype=tf.int8)

In [13]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi        13Gi       383Mi       597Mi       1.6Gi       978Mi
Swap:           31Gi       2.7Gi        29Gi


In [14]:
sys.getsizeof(X)

184

In [15]:
del X

In [16]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       9.9Gi       4.1Gi       592Mi       1.6Gi       4.6Gi
Swap:           31Gi       2.7Gi        29Gi


**(?)** Why my 4GB-RAM Thinkpad X61s still unable to allocate for this `X` using `tf`? Isn't that allocation just a mere `184` bytes?

Note that the `free` commands above showed that the RAM consumption equals approximately
- `  0GB` for the case of `np.zeros(dtype=np.float32)`
- `  1GB` for the case of `tf.zeros(dtype=tf.int8)`
- `3.7GB`for the case of `tf.zeros(dtype=tf.float32)`

So, even though `sys.getsizeof(X)` shows less in `tf` tensor than in `np` ndarray, the OS feels the other way around.<br>
**Suspicious...**